In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [ ]:
llm.invoke("How will the weather be in munich today?")

In [25]:
from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


tools = [get_weather]

In [26]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

In [ ]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

In [ ]:
result.tool_calls

In [30]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [ ]:
messages

In [32]:
tool_mapping = {
    "get_weather": get_weather
}

In [33]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages)